# Purpose:
- Assume data is searched and loaded (develop this part later)
    - Use data from "Saffron" (721291)
- Run design matrix, per session
- Test attaching ophys data as well (consider memory limit)

In [258]:
from DesignMatrix import DesignMatrix
from comb.behavior_ophys_dataset import BehaviorOphysDataset, BehaviorMultiplaneOphysDataset
from comb.behavior_session_dataset import BehaviorSessionDataset
import os
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr

# notebook dev
%load_ext autoreload
%autoreload 2
%matplotlib inline
import capsule_utils
import load_data
import design_matrix_tools as dmtools
import kernel_tools as ktools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [256]:
session_name = 'multiplane-ophys_721291_2024-05-16_08-57-00'

data_dir = Path('/root/capsule/data/')
raw_path = data_dir / session_name
processed_path = list(data_dir.glob(f'{session_name}_processed*'))[0]

opids = []
for plane_folder in processed_path.glob("*"):
    if plane_folder.is_dir() and not plane_folder.stem.startswith("nextflow"):
        opid = plane_folder.stem
        opids.append(opid)

bod_list = []
for opid in opids:
    bod = load_data.load_plane_data(session_name, opid=opid)
    bod = capsule_utils.add_trials_to_bod(bod)
    bod_list.append(bod)
bod = bod_list[0]

File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


...

/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


In [247]:
def define_kernels():
    kernels = {
        'intercept':    {'feature':'intercept',   'type':'continuous',    'length':0,     'offset':0,     'num_weights':None, 'dropout':True, 'text': 'constant value'},
        'hits':         {'feature':'hit',         'type':'discrete',      'length':1.5,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'lick to image change'},
        'misses':       {'feature':'miss',        'type':'discrete',      'length':1.5,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'no lick to image change'},
        'passive_change':   {'feature':'passive_change','type':'discrete','length':1.5,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'passive session image change'},
        #'hits':         {'feature':'hit',         'type':'discrete',      'length':.75,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'lick to image change'},
        #'misses':       {'feature':'miss',        'type':'discrete',      'length':.75,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'no lick to image change'},
        #'passive_change':   {'feature':'passive_change','type':'discrete','length':.75,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'passive session image change'},
        #'post-hits':    {'feature':'hit',         'type':'discrete',      'length':1.5,   'offset':0.75,    'num_weights':None, 'dropout':True, 'text': 'lick to image change'},
        #'post-misses':  {'feature':'miss',        'type':'discrete',      'length':1.5,   'offset':0.75,    'num_weights':None, 'dropout':True, 'text': 'no lick to image change'},
        #'post-passive_change': {'feature':'passive_change','type':'discrete','length':1.5,   'offset':0.75,    'num_weights':None, 'dropout':True, 'text': 'passive session image change'},
        'omissions':        {'feature':'omissions',   'type':'discrete',  'length':1.5,      'offset':0,     'num_weights':None, 'dropout':True, 'text': 'image was omitted'},
        #'omissions':        {'feature':'omissions',   'type':'discrete',  'length':0.75,      'offset':0,     'num_weights':None, 'dropout':True, 'text': 'image was omitted'},
        #'post-omissions':   {'feature':'omissions',   'type':'discrete',  'length':2.25,   'offset':0.75,  'num_weights':None, 'dropout':True, 'text': 'images after omission'},
        'each-image':   {'feature':'each-image',  'type':'discrete',      'length':0.75,  'offset':0,     'num_weights':None, 'dropout':True, 'text': 'image presentation'},
        'running':      {'feature':'running',     'type':'continuous',    'length':2,     'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'normalized running speed'},
        # 'pupil':        {'feature':'pupil',       'type':'continuous',    'length':2,     'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'Z-scored pupil diameter'},
        'licks':        {'feature':'licks',       'type':'discrete',      'length':2,     'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'mouse lick'},
        #'false_alarms':     {'feature':'false_alarm',   'type':'discrete','length':5.5,   'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'lick on catch trials'},
        #'correct_rejects':  {'feature':'correct_reject','type':'discrete','length':5.5,   'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'no lick on catch trials'},
        #'time':         {'feature':'time',        'type':'continuous',    'length':0,     'offset':0,    'num_weights':None,  'dropout':True, 'text': 'linear ramp from 0 to 1'},
        #'beh_model':    {'feature':'beh_model',   'type':'continuous',    'length':.5,    'offset':-.25, 'num_weights':None,  'dropout':True, 'text': 'behavioral model weights'},
        #'lick_bouts':   {'feature':'lick_bouts',  'type':'discrete',      'length':4,     'offset':-2,   'num_weights':None,  'dropout':True, 'text': 'lick bout'},
        #'lick_model':   {'feature':'lick_model',  'type':'continuous',    'length':2,     'offset':-1,   'num_weights':None,  'dropout':True, 'text': 'lick probability from video'},
        #'groom_model':  {'feature':'groom_model', 'type':'continuous',    'length':2,     'offset':-1,   'num_weights':None,  'dropout':True, 'text': 'groom probability from video'},
    }
    ## add face motion energy PCs
    # for PC in range(5):
    #     kernels['face_motion_PC_{}'.format(PC)] = {'feature':'face_motion_PC_{}'.format(PC), 'type':'continuous', 'length':2, 'offset':-1, 'dropout':True, 'text':'PCA from face motion videos'}
    
    return kernels

In [260]:
run_params = {'data_type': 'filtered_events'}
kernels = define_kernels()


response_list = []
run_params_list = []
for bod in bod_list:
    run_params = ktools.process_kernels(kernels, run_params, bod)
    response, run_params = load_data.extract_and_annotate_ophys_plane(bod, run_params)
    response_list.append(response)
    run_params_list.append(run_params)

Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps


In [261]:
responses = response_list[0]
design = DesignMatrix(response)
dmtools.add_kernels(design, run_params, bod, response)

    Adding kernel: intercept
    Adding kernel: hits
    Adding kernel: misses
    Adding kernel: passive_change
	Error encountered while adding kernel for passive_change. Attemping to continue without this kernel.
	Passive Change kernel cant be added to active sessions
    Adding kernel: omissions
    Adding kernel: running
                 : Mean Centering
                 : Standardized to unit variance
    Adding kernel: licks
    Adding kernel: im000
    Adding kernel: im031
    Adding kernel: im035
    Adding kernel: im045
    Adding kernel: im054
    Adding kernel: im073
    Adding kernel: im075
    Adding kernel: im106
The following kernels failed to be added to the model: 
{'passive_change'}

The following dropouts failed to be added to the model: 
{'single-passive_change'}



In [262]:
X = design.get_X()
assert X.shape[0] == response['response_arr'].shape[0]